In [10]:
from dolfinx import default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, assemble_scalar,
                         form, locate_dofs_geometrical, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import create_unit_square, locate_entities, locate_entities_boundary
from dolfinx.plot import vtk_mesh

import ufl
from ufl import (SpatialCoordinate, TestFunction, TrialFunction,
                 dx, grad, inner)

from mpi4py import MPI

import meshio
# import gmsh
import numpy as np
import pyvista

pyvista.start_xvfb()

In [11]:
# 材料应力应变
E, nu = 1.0, 1.0 / 3.0

def strain(u):
    return 1.0/2.0*(ufl.grad(u) + ufl.grad(u).T)

def sigma(u):
    return E / (1.0 - nu**2) * ((1.0 - nu) * strain(u) + nu * ufl.Identity(2) * ufl.tr(strain(u)))

In [12]:
# 10x10单位正方形，节点数量11x11
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)
V = functionspace(mesh, ('Lagrange', 1))
u = TrialFunction(V)
v = TestFunction(V)

In [15]:
# 边界条件
# 狄利克雷边界条件
fdim = mesh.topology.dim - 1
def dirichlet_(x):
    return np.isclose(x[0], 0)


# locate_entities_boundary返回ndarry，
dirichelet_facet = locate_entities_boundary(mesh, fdim, dirichlet_)
print(dirichelet_facet.shape)



(10,)
